# Extracción de recuerdos de videos



In [1]:
#@title # Instalaciones
#@markdown Nota: es necesario reiniciar el runtime tras instalar para poder usar las instalaciones. El avisó saldrá debajo de la celda con un botón "Restart Runtime".

!pip install spacy==3.2.3
!pip install pyyaml==6.0
!python -m spacy download es_core_news_lg
!python -m spacy download es_dep_news_trf
!pip install networkx==2.6.3
!pip install neo4j==4.4.2
!pip install --upgrade google-cloud-speech

!python -m nltk.downloader wordnet_ic
!python -m nltk.downloader omw
!python -m nltk.downloader wordnet

     |████████████████████████████████| 6.0 MB 6.8 MB/s 
     |████████████████████████████████| 10.1 MB 28.7 MB/s 
     |████████████████████████████████| 653 kB 49.2 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 181 kB 71.7 MB/s 
     |████████████████████████████████| 457 kB 45.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzi

In [1]:
#@title # Montar Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title # Establecer directorio de trabajo

import os
path_carpeta_ejecucion = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab' #@param {type:"string"}
#@markdown Consejo: En el explorador de archivos de Google Colab puedes obtener el path a una carpeta o archivo haciendo click derecho y seleccionando "copiar ruta"
path_carpeta_ejecucion.encode()
os.chdir(path_carpeta_ejecucion)

In [3]:
#@title # Módulo de transcripción

import os
import moviepy.editor as mp
from google.cloud import storage
from google.cloud import speech_v1p1beta1 as speech

def uploadAudio(file, fileFormat, credentialsPath, bucketName):
    # Load credentials in enviromental variable
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentialsPath
    print('credentials set')
    
    # Connect to the bucket
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucketName)
    print('bucket connected')
    blob = bucket.blob('tempAudio.mp3')
    
    # Transform video to audio
    if fileFormat:
        # If file is video
        clip = mp.VideoFileClip(file)
        clip.audio.write_audiofile(r"tempAudio.mp3")
    else:
        # If file is audio
        clip = mp.AudioFileClip(file)
        clip.write_audiofile(r"tempAudio.mp3")
    print('audio created')
    
    # Upload audio to bucket
    blob.upload_from_filename('tempAudio.mp3')
    print('audio uploaded to bucket')
    
    # Delete audio file
    os.remove(r"tempAudio.mp3")
    print('audio deleted')
    
def transcribeAudio(bucketName):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    gcs_uri = 'gs://' + bucketName + '/tempAudio.mp3'
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig( 
        language_code = 'es-ES',
        enable_speaker_diarization=True,
        diarization_speaker_count=2,
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        enable_automatic_punctuation=True,
        sample_rate_hertz=16000)

    response = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    result = response.result().results[-1]

    words_info = result.alternatives[0].words

    # Printing out the output:
    transcription = []
    curr_s = 0
    curr_t = ''
    for word_info in words_info:
        word_info.word, word_info.speaker_tag
        if curr_s != word_info.speaker_tag:
            transcription.append(curr_t)
            curr_s = word_info.speaker_tag
            curr_t = str(word_info.speaker_tag) + ': ' + word_info.word
        else:
            curr_t = curr_t + ' ' + word_info.word
    transcription.append(curr_t)
    
    # Connect to the bucket
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucketName)
    blob = bucket.blob('tempAudio.mp3')
    blob.delete()
    print('bucket object deleted')
    return transcription

In [4]:
#@title # Módulo auxiliar de extracción de grafos

#SPACY
import spacy
from spacy.matcher import Matcher

#create  nlp object
nlp = spacy.load("es_core_news_lg")

#GRAFENO
from grafeno import pipeline
from grafeno.transformers import get_pipeline
from grafeno.jupyter import visualize
import yaml

#NEO4J
from neo4j import GraphDatabase

import os

#checks if two texts are similar
def similar(txt1, txt2):
    doc1 = nlp(txt1.lower().replace('¿', '').replace('?', ''))
    doc2 = nlp(txt2.lower().replace('¿', '').replace('?', ''))
    #print(txt1)
    #print(txt2)
    #print('SIMILITUD: ',doc1.similarity(doc2))
    if doc1.similarity(doc2) > 0.8:
        return doc1.similarity(doc2)
    else:
        sep = txt2.split(", ")
        for s in sep:
            s = doc1.similarity(nlp(s))
            if s > 0.8:
                return s
        return 0

def createGraphQuery(text, questionNumber):
    # Load Grafeno pipeline config
    config = yaml.safe_load(open(os.getcwd() + '/configs/custom.yaml'))
    
    # Run the pipeline on the text to obtain Neo4j query representin the graph obtained
    query = pipeline.run({ **config, 'text': text })
    
    return(query.replace('PX', 'P' + str(questionNumber)))

def createDateQuery(text, questionNumber):
    lines = text.splitlines()
    answer = ''
    if len(lines) > 1:
        for i in range(1, len(lines)):
            answer = answer + '. ' + lines[i]
    # Create doc
    doc = nlp(answer)

    # Initialize the matcher with the shared vocab
    matcher = Matcher(nlp.vocab)

    # Add date recognition patterns to the matcher
    # Year pattern (from 1900 to 2199)
    YearPattern = [[{"TEXT": {"REGEX": "(19|20|21)\d\d"}}]]
    # Day of the month pattern (ex: 23 de enero)
    DayMonthPattern = [[{"IS_DIGIT": True}, {"LOWER": "de"}, {"LOWER": {"REGEX": "^(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)$"}}]]
    # Month pattern (from enero to diciembre)
    MonthPattern = [[{"LOWER": {"REGEX": "^(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)$"}}]]
    # Day of the week pattern (from lunes to martes)
    DayPattern = [[{"LOWER": {"REGEX": "^(lunes|martes|miercoles|jueves|viernes|sabado|domingo)$"}}]]
    matcher.add("year", YearPattern)
    matcher.add("day", DayMonthPattern)
    matcher.add("month", MonthPattern)
    matcher.add("dayWeek", DayPattern)
    
    # Call the matcher on the doc
    matches = matcher(doc)

    # Iterate over the matches to find dates
    dates = {}
    for match_id, start, end in matches:
        # Get the matched span
        matched_span = doc[start:end]
        dates[doc.vocab.strings[match_id]] = matched_span.text
        
    # Insert matches into Neo4j cypher query
    # It merges a node with the matches as properties
    # Creates a relationship with the verb node of the same question, if it exists
    query = ''
    if dates and len(dates)>0:
        query = ('MERGE (n:DATE {' +', '.join('{}: {!r}'.format(k, str(dates[k])) for k in dates) +
                '}) SET n:PX, n._temp_id=\'0\';\n' + 
                'MATCH (n:PX:VERB), (m {_temp_id: \'0\'}) CREATE (n)-[r:CONTEXT]->(m);\n' +
                'MATCH (n) REMOVE n._temp_id;')
    return createGraphQuery(lines[0], questionNumber) + '\n' + query.replace('PX', 'P' + str(questionNumber))
    
def createTextQuery(text, questionNumber):
    lines = text.splitlines()
    answer = ''
    if len(lines) > 1:
        for i in range(1, len(lines)):
            answer = answer + '. ' + lines[i]
    query = 'CREATE (n:TEXT:PX {{concept: \'{}\'}});'.format(answer);
    return query.replace('PX', 'P' + str(questionNumber))

#produces the results based on the mode
def process(text, questionNumber, mode):
    if mode == 0: return createGraphQuery(text, questionNumber)
    elif mode == 1: return createDateQuery(text, questionNumber)
    elif mode == 2: return createTextQuery(text, questionNumber)

    


<IPython.core.display.Javascript object>

In [5]:
#@title # Módulo de extracción de grafos

import json

def extractGraphs(lines, scriptPath):
    #load questions and results modes on dict
    with open(scriptPath, encoding='utf-8') as f:
        script = json.load(f)['preguntas']

    #get questions
    questions = list(script.keys())
    results = {}

    # Find the positions of the questions in the text lines, using similarity
    matches = [-1] * len(lines)
    for j in range(len(questions)):
        maxS = 0
        maxI = -1
        for i in range(len(lines)):
            if matches[i] == -1:
                s = similar(questions[j], lines[i][2:])
                if s > maxS and s > 0.7:
                    maxS = s
                    maxI = i
        if maxI != -1:
            matches[maxI] = j

    segment = ''
    queries = []
    segments = []

    # Process each question to obtain queries
    qNum = -1
    for i in range(len(lines)):
        if qNum != -1:
            if matches[i] != -1:
                segments.append(segment)
                queries.append(process(segment, qNum, script[questions[qNum]]))
                #print(segment + '\n\n')
                qNum = matches[i]
                segment = lines[i][2:]
            else:
                segment = segment + '\n' + lines[i][2:]

        elif matches[i] != -1:
            qNum = matches[i]
            segment = lines[i][2:]
        i += 1
    segments.append(segment)
    queries.append(process(segment, j, script[questions[j]]))
    #print(segment + '\n\n')    
    return (queries, segments)

def submitQueries(queries, user, pwd, uri):
    data_base_connection = GraphDatabase.driver(uri = uri, auth=(user, pwd))
    session = data_base_connection.session()   
    for qs in queries:
        for q in qs.splitlines():
            session.run(q)
    session.run('match ()-[r]->() match (s)-[r]->(e) with s,e,type(r) as typ, tail(collect(r)) as coll foreach(x in coll | delete x);')


In [6]:
#@title # Opciones de transcripción:

modo_transcripcion = 'Automatica' #@param ["Automatica", "Manual"]

mostrar_transcripcion = True #@param {type: "boolean"}

#@markdown ---
#@markdown ### Modo manual:

texto = "2: Hola, qu\xE9 tal est\xE1s muy bien, c\xF3mo te llamas? 1: Yo soy Jos\xE9 Miguel 2: te puedo llamar Jos\xE9 1: tambi\xE9n me puedes llamar Josemi 2: cu\xE1ntos a\xF1os tiene Jos\xE9 65, en qu\xE9 a\xF1o naciste 1: en 1956 el 2 de febrero 2: en d\xF3nde 1: en Castilla y Le\xF3n m\xE1s concretamente en" #@param {type:"string"}
texto.encode()
texto2 = texto.replace('2:', '\n2:').replace('1:', '\n1:')
#@markdown ---
#@markdown ### Modo automático:

formato_archivo = 'Video' #@param ["Video", "Audio"]

path_archivo = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab/videoC.mov' #@param {type:"string"}

path_clave = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab/tfg-2021-2022-storage.json' #@param {type:"string"}

nombre_bucket = 'bucket-tfg-2021-2022' #@param {type:"string"}

In [7]:
#@title # Ejecutar Transcripción

if modo_transcripcion == 'Manual':
    transcription = texto2.splitlines()
elif modo_transcripcion == 'Automatica':
    uploadAudio(path_archivo, formato_archivo == 'Audio',  path_clave, nombre_bucket)
    transcription = transcribeAudio(nombre_bucket)
if mostrar_transcripcion:
    print('\033[1m' + 'Transcripción: \n' + '\033[0m')
    for t in transcription:
        print(t)

credentials set
bucket connected
[MoviePy] Writing audio in tempAudio.mp3


100%|██████████| 3539/3539 [00:04<00:00, 789.28it/s]

[MoviePy] Done.


audio created
audio uploaded to bucket
audio deleted
Waiting for operation to complete...
bucket object deleted
Transcripción: 


2: Hola, qué tal estás muy bien, cómo te llamas?
1: Yo soy José Miguel
2: te puedo llamar José
1: también me puedes llamar Josemi
2: cuántos años tiene José 65, en qué año naciste
1: en 1956 el 2 de febrero
2: en dónde
1: en Castilla y León más concretamente en
2: Salamanca aprenderán tus padres
1: uno de allí mismo de Salamanca y otro
2: cuántos hermanos
1: tuviste 4 hermanos 2 chicos y 2 chicas se llamaban Fernando Luis María Eugenia yo el segundo más pequeño,
2: cómo era la relación con tus hermanos
1: yo me llevaba muy bien con mis hermanos todos en general, nos hemos llevado bien, pero yo me lleva mejor con los chicos cuando era pequeño. A veces nos peleamos bueno, que te voy a contar a ti lo típico,
2: estás fotocuentos, qué aficiones tenías?
1: Pues jugamos al fútbol, siempre jugado en algún equipo era delante y metía muchos goles,
2: cuáles son los m

In [8]:
#@title # Opciones de extracción de grafos:


guion = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab/guion_entrevista.json' #@param {type:"string"}

mostrar_segmentos = True #@param {type:"boolean"}
mostrar_consultas = True #@param {type:"boolean"}
subir_neo4j = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Neo4j:

usuario = 'neo4j' #@param {type:"string"}
contraseña = 'passbook-connection-lieutenants' #@param {type:"string"}
uri = 'bolt://44.201.103.117:7687' #@param {type:"string"}

In [9]:
#@title # Ejecutar extracción de grafos

results = extractGraphs(transcription, guion)
if mostrar_segmentos or mostrar_consultas:
    for i in range(len(results[0])):
            print('\033[1m' + 'Pregunta ' + str(i+1) + '\033[0m' + '\n')
            if mostrar_segmentos: print('\033[94m' + 'Segmento:  \n' + results[1][i] + '\033[0m'+ '\n')
            if mostrar_consultas: print('\033[36m' + 'Consulta:  \n' + results[0][i] + '\033[0m' + '\n')
if subir_neo4j:
    submitQueries(results[0], usuario, contraseña, uri)

Pregunta 1

Segmento:  
 Hola, qué tal estás muy bien, cómo te llamas?
 Yo soy José Miguel
 te puedo llamar José
 también me puedes llamar Josemi

Consulta:  
MERGE (n:NOUN {concept: 'José', sempos: 'n', proper: 'True', num: 'p'}) SET n:P0, n._temp_id='0';
MERGE (n:ADVERB {concept: 'bien', sempos: 'r', synset: "Synset('well.r.13')"}) SET n:P0, n._temp_id='1';
MERGE (n:ADVERB {concept: 'mucho', sempos: 'r', synset: "Synset('much.r.01')"}) SET n:P0, n._temp_id='2';
MERGE (n:VERB {concept: 'llamar', sempos: 'v', synset: "Synset('call.v.03')"}) SET n:P0, n._temp_id='3';
MERGE (n:NOUN {concept: 'Miguel', sempos: 'n', proper: 'True', num: 'p', synset: "Synset('michael.n.01')"}) SET n:P0, n._temp_id='4';
MERGE (n:ADVERB {concept: 'también', sempos: 'r', synset: "Synset('besides.r.02')"}) SET n:P0, n._temp_id='5';
MERGE (n:NOUN {concept: 'Josemi', sempos: 'n', proper: 'True', num: 'p'}) SET n:P0, n._temp_id='6';
MATCH (n {_temp_id: '0'}), (m {_temp_id: '4'}) CREATE (n)-[r:ATTR]->(m);
MATCH (n 